<a href="https://colab.research.google.com/github/Anjasfedo/Learning-TensorFlow/blob/main/learn_by_doing/Police_Arrest_from_2021_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download datasets

In [2]:
!pip install kaggle

from google.colab import files
files.upload

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d shayanshahid997/police-arrest-from-2021-2023

!unzip police-arrest-from-2021-2023.zip

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/shayanshahid997/police-arrest-from-2021-2023
License(s): CC0-1.0
police-arrest-from-2021-2023.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  police-arrest-from-2021-2023.zip
  inflating: Police_Arrests_20240702.csv  


# Initialize Packages

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers, models

# Preprocessing Dataset

In [8]:
df = pd.read_csv('Police_Arrests_20240702.csv')
df

,ID Reference Number,Subject's race,Subject's gender,Subject's age,Ethnicity,District of occurrence,Adjacent to School,Assigned Division,Assigned Bureau,Event Date/Time
0,1,Asian,Male,43,NON-HISPANIC,6.0,0,TOD,FSB,1/1/2021 1:07
1,2,Black/African American,Male,23,NON-HISPANIC,1.0,0,RCPD,NaN,1/1/2021 0:52
2,3,Black/African American,Male,18,NON-HISPANIC,1.0,0,RCPD,NaN,1/1/2021 0:52
3,4,Black/African American,Male,21,NON-HISPANIC,1.0,0,RCPD,NaN,1/1/2021 0:52
4,5,White,Female,38,HISPANIC,4.0,0,TOD,FSB,1/1/2021 3:00
...,...,...,...,...,...,...,...,...,...,...
23505,23506,Black/African American,Male,30,NON-HISPANIC,3.0,0,3D,PSB,6/30/2024 11:00
23506,23507,Asian,Male,60,NON-HISPANIC,3.0,0,3D,PSB,7/1/2024 7:27
23507,23508,White,Male,22,HISPANIC,4.0,0,2D,PSB,6/29/2024 2:42
23508,23509,White,Male,25,HISPANIC,4.0,0,2D,PSB,6/29/2024 2:42


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23510 entries, 0 to 23509
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID Reference Number     23510 non-null  int64  
 1   Subject's race          23510 non-null  object 
 2   Subject's gender        23510 non-null  object 
 3   Subject's age           23510 non-null  int64  
 4   Ethnicity               23510 non-null  object 
 5   District of occurrence  23408 non-null  float64
 6   Adjacent to School      23510 non-null  int64  
 7   Assigned Division       23381 non-null  object 
 8   Assigned Bureau         18851 non-null  object 
 9   Event Date/Time         23510 non-null  object 
dtypes: float64(1), int64(3), object(6)
memory usage: 1.8+ MB


In [11]:
df.describe()

,ID Reference Number,Subject's age,District of occurrence,Adjacent to School
count,23510.00000,23510.000000,23408.000000,23510.000000
mean,11755.50000,31.813143,3.989192,0.085155
std,6786.89675,12.946909,1.748587,0.279119
min,1.00000,1.000000,1.000000,0.000000
25%,5878.25000,22.000000,3.000000,0.000000
50%,11755.50000,30.000000,4.000000,0.000000
75%,17632.75000,39.000000,5.000000,0.000000
max,23510.00000,99.000000,8.000000,1.000000


In [9]:
df.isnull().sum()

ID Reference Number          0
Subject's race               0
Subject's gender             0
Subject's age                0
Ethnicity                    0
District of occurrence     102
Adjacent to School           0
Assigned Division          129
Assigned Bureau           4659
Event Date/Time              0
dtype: int64

In [46]:
# Find rows where at least one column contains null values
columns_with_null = df.columns[df.isnull().any()]

columns_with_null

Index(['District of occurrence', 'Assigned Division', 'Assigned Bureau'], dtype='object')

In [58]:
for column in columns_with_null:
  print(df[column].unique())

[ 6.  1.  4.  3.  5.  2.  8. nan]
['TOD' 'RCPD' 'TPPD' 'GPD' '4D' '3D' '5D' '6D' 'IMTD' '2D' '1D' 'SVID'
 'CID' 'MCFM' 'SID' 'PSTA' nan 'MCSO' 'MCD' 'SOD' 'CED' 'PSCC' 'FSB'
 'PERS' 'ISB']
['FSB' nan 'PSB' 'MSB' 'ISB']


In [60]:
sampled_dfs = []

for column in columns_with_null:
    rows_with_null_in_column = df[df[column].isnull()]

    sample_rows_with_null = rows_with_null_in_column.sample(2)  # Adjust sample size as needed

    sample_rows_with_null['Null Column'] = column

    sampled_dfs.append(sample_rows_with_null)

# Concatenate all sampled DataFrames into one DataFrame
sampled_df = pd.concat(sampled_dfs, ignore_index=True)

# Print the concatenated DataFrame
print("Sample of rows with null values:")
print(sampled_df)

Sample of rows with null values:
   ID Reference Number           Subject's race Subject's gender  \
0                  8491  Black/African American             Male   
1                 17656  Black/African American             Male   
2                 12464                   White             Male   
3                 10111                   White             Male   
4                 11751  Black/African American             Male   
5                 10551  Black/African American             Male   

   Subject's age     Ethnicity  District of occurrence  Adjacent to School  \
0             20  NON-HISPANIC                     NaN                   0   
1             14  NON-HISPANIC                     NaN                   0   
2             20  NON-HISPANIC                     2.0                   0   
3             15      HISPANIC                     4.0                   0   
4             30  NON-HISPANIC                     6.0                   0   
5             54  NON-

Column information:
- ID Reference Number          : unique identifier for each arrest record (discard)
- Subject's race               : demographic information
- Subject's gender             : demographic information
- Subject's age                : demographic information (number)
- Ethnicity                    : demographic information
- District of occurrence       : indicates the district in which the arrest took place (missing, number)
- Adjacent to School           : whether the arrest took place within 500 feet of a school, with a 1 indicating that it did and a 0 indicating that it did not (number)
- Assigned Division            : division and bureau of the police department that were assigned to the arrest (missing)
- Assigned Bureau              : division and bureau of the police department that were assigned to the arrest (missing)
- Event Date/Time              : provides the date and time of the arrest

In [67]:
def preprocessing(df_data):
  df_result = pd.DataFrame()

  # Race
  df_subjects_race = pd.get_dummies(df_data['Subject\'s race'], prefix='Subject race').astype('int32')
  df_result = pd.concat([df_result, df_subjects_race], axis=1)

  # Gender
  df_subjects_gender = pd.get_dummies(df_data['Subject\'s gender'], prefix='Subject gender').astype('int32')
  df_result = pd.concat([df_result, df_subjects_gender], axis=1)

  # Age
  df_result['Subject\'s age'] = df_data['Subject\'s age'].astype('int32')

  # Ethnicity
  df_ethnicity = pd.get_dummies(df['Ethnicity'], prefix='Ethnicity').astype('int32')
  df_result = pd.concat([df_result, df_ethnicity], axis=1)

  # District of occurrence
  df_result['District of occurrence'] = df_data['District of occurrence'].fillna(0).astype('int32')
  df_result['District of occurrence null'] = pd.isna(df_data['District of occurrence']).astype('int32')

  # Adjacent to School
  df_result['Adjacent to School'] = df_data['Adjacent to School'].astype('int32')

  # # Assigned Division
  df_assigned_division = pd.get_dummies(df_data['Assigned Division'], dummy_na=True, prefix='Assigned Division').astype('int32')
  df_result = pd.concat([df_result, df_assigned_division], axis=1)

  # Assigned Bureau
  df_assigned_bureau = pd.get_dummies(df_data['Assigned Bureau'], dummy_na=True, prefix='Assigned Bureau').astype('int32')
  df_result = pd.concat([df_result, df_assigned_bureau], axis=1)

  return (df_result)

In [68]:
df_result = preprocessing(df)
df_result

,Subject race_American Indian/ALSK Natv,Subject race_Asian,Subject race_Black/African American,Subject race_Hawaiian/Pacific Islander,Subject race_Unknown,Subject race_White,Subject gender_Female,Subject gender_Male,Subject's age,Ethnicity_HISPANIC,...,Assigned Division_SOD,Assigned Division_SVID,Assigned Division_TOD,Assigned Division_TPPD,Assigned Division_nan,Assigned Bureau_FSB,Assigned Bureau_ISB,Assigned Bureau_MSB,Assigned Bureau_PSB,Assigned Bureau_nan
0,0,1,0,0,0,0,0,1,43,0,...,0,0,1,0,0,1,0,0,0,0
1,0,0,1,0,0,0,0,1,23,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,1,0,0,0,0,1,18,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,1,0,0,0,0,1,21,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,1,1,0,38,1,...,0,0,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23505,0,0,1,0,0,0,0,1,30,0,...,0,0,0,0,0,0,0,0,1,0
23506,0,1,0,0,0,0,0,1,60,0,...,0,0,0,0,0,0,0,0,1,0
23507,0,0,0,0,0,1,0,1,22,1,...,0,0,0,0,0,0,0,0,1,0
23508,0,0,0,0,0,1,0,1,25,1,...,0,0,0,0,0,0,0,0,1,0
